In [ ]:
! pip install langchain_openai langchain_community langchain_core chromadb -U

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from google.colab import userdata
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

import os

os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

def queryFromRag(question):
    loader = WebBaseLoader(
        web_path="https://baike.baidu.com/item/%E6%B5%B7%E6%B7%80%E5%8C%BA?fromtitle=%E6%B5%B7%E6%B7%80&fromid=5104876&fromModule=lemma_search-box",
    )

    embedding = OpenAIEmbeddings(model="text-embedding-v3", check_embedding_ctx_length=False)

    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    db = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory="./chroma_db")

    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})
    search = retriever.invoke(input=question)

    search_str = " ".join([doc.page_content for doc in search]) 

    llm = ChatOpenAI(model="qwen2-72b-instruct", temperature=0)

    res = llm.invoke([
        SystemMessage("你是一个智能助手，请根据上下文回答问题，上下文是:\n" + search_str),
        HumanMessage(question)
    ])
    
    return res.content


result = queryFromRag("海淀区常住人口是多少")

print(result)
